In [4]:
import numpy as np 
from functools import wraps 
from time import time

<H4> Option Inputs <H4>

In [2]:
# Initialise parameters
S0 = 100        # inital stock price
K = 100         # strike price
T = 1           # time to maturity in years
r = 0.045       # annual risk-free rate
N = 3           # Number of time steps 
u = 1.1         # up-factor in binomal models
d = 1/u         # ensure recombining tree
opttype = "P"   #Option Type 'C' or 'P'
        

<h3> American Tree Slow <h3>
<h5>We will use for loops to iterate through nodes j at each step i<h5>

In [5]:
def american_slow_tree(K,T,S0,r,N,u,d,opttype = "P"):
    # precompute values 
    dt = T/N
    q =(np.exp(r*dt) - d)/ (u-d)
    disc =  np.exp(-r*dt)
    
    # intialize stock prices at maturity 
    S = np.zeros(N+1)
    for j in range(0, N+1):
        S[j] = S0 * u **j * d**(N-j)
    
    # option payoff
    
    C = np.zeros(N+1)
    for j in range(0, N+1):
        if opttype == "P":
            C[j] = max(0, K - S[j])
        else:
            C[j] = max(0, S[j] - K)
            
    # backward recuring through the tree
    
    for i in np.arange(N -1,-1,-1):
        for j in range(0,i +1):
            S = S0 * u ** j * d**(i-j)
            C[j] = disc * (q* C[ j +1 ]  + (1-q) * C[j])
            if opttype == "P":
                C[j] = max(C[j], K - S)
            else:
                C[j] = max(C[j], S - K)
    
    return C[0]
american_slow_tree(K,T,S0,r,N,u,d,opttype = "P")

5.238420097499048

<H3> Fast Version Using Numpy Arrays<H3>

In [6]:
def american_fast_tree(K,T,S0,r,N,u,d,opttype = "P"):
    # precompute values 
    dt = T/N
    q = (np.exp(r*dt) - d)/ (u-d)
    disc =  np.exp(-r*dt)
    
    # intialize stock prices at maturity 
    S = S0 * d**(np.arange(N, -1,-1)) * u **(np.arange(0,N+1,1))
    
    # option payoff
    if opttype == "P":
        C = np.maximum(0, K - S)
    else:
        C = np.maximum(0, S - K)
            
    # backward recuring through the tree
    
    for i in np.arange(N -1,-1,-1):
        S = S0 * d**(np.arange(i, -1,-1)) * u **(np.arange(0,i +1,1))
        C[:i+1] = disc * (q* C[1:i+2] + (1-q) * C[0:i +1])
        C = C[:-1]
        if opttype == "P":
            C = np.maximum(C, K - S)
        else:
            C = np.maximum(C, S - K)
    return C[0]

american_fast_tree(K,T,S0,r,N,u,d,opttype = "P")

5.238420097499048

In [36]:
# Initialise parameters
S0 = 100        # inital stock price
K = 100         # strike price
T = 1           # time to maturity in years
r = 0.06       # annual risk-free rate
N = 3           # Number of time steps 
u = 1.1         # up-factor in binomal models
d = 1/u
sigma = 0.3        # ensure recombining tree
opttype = "P"   #Option Type 'C' or 'P'

<H3> Cox, Ross and Rubinstien (CRR) Method <H3>

In [33]:
def CRR_method(K,T,S0,r,N,sigma,opttype = "P"):
    # precompute values 
    dt = T/N
    u = np.exp(sigma* np.sqrt(dt))
    d = 1/u
    q =(np.exp(r*dt) - d)/ (u-d)
    disc =  np.exp(-r*dt)
    
    # intialize stock prices at maturity 
    S = np.zeros(N+1)
    S[0] = S0 * d** N
    for j in range(0, N+1):
        S[j] = S[j-1]*u/d
    
    # option payoff
    
    C = np.zeros(N+1)
    for j in range(0, N+1):
        if opttype == "P":
            C[j] = max(0, K - S[j])
        else:
            C[j] = max(0, S[j] - K)
            
    # backward recuring through the tree
    
    for i in np.arange(N -1,-1,-1):
        for j in range(0,i +1):
            S = S0 * u ** j * d**(i-j)
            C[j] = disc * (q* C[ j +1 ]  + (1-q) * C[j])
            if opttype == "P":
                C[j] = max(C[j], K - S)
            else:
                C[j] = max(C[j], S - K)
    
    return C[0]

CRR_method(K,T,S0,r,N,sigma,opttype = "P")

10.25146322970778

In [38]:
def JR_method(K,T,S0,r,N,sigma,opttype = "C"):
    # precompute values 
    dt = T/N
    nu = r - 0.5* sigma **2
    u = np.exp(nu * dt + sigma*np.sqrt(dt))
    d = np.exp(nu * dt - sigma*np.sqrt(dt))
    q = 0.5
    disc =  np.exp(-r*dt)
    
    # intialize stock prices at maturity 
    S = np.zeros(N+1)
    S[0] = S0 * d ** N
    for j in range(0, N+1):
        S[j] = S[j-1]*u/d
    
    # option payoff
    
    C = np.zeros(N+1)
    for j in range(0, N+1):
        if opttype == "C":
            C[j] = max(0, K - S[j])
        else:
            C[j] = max(0, S[j] - K)
            
    # backward recuring through the tree
    
    for i in np.arange(N -1,-1,-1):
        for j in range(0,i +1):
            S = S0 * u ** j * d**(i-j)
            C[j] = disc * (q* C[ j +1 ]  + (1-q) * C[j])
    
    return C[0]

JR_method(K,T,S0,r,N,sigma,opttype = "C")

94.17645335842487